# Molecular Dynamics Simulation in implicit solvent

Let's create a Langevin MD trajectory 500 ns long, saved every 5 ps, in implicit solvent with OpenMM.

In [1]:
import molsysmt as msm
import openmm as omm
import openmm.app as app
from openmm import unit
from mdtraj.reporters import HDF5Reporter
from sys import stdout

In [2]:
pdb = app.PDBFile('trp_cage.pdb')

In [3]:
forcefield = app.ForceField('amber14-all.xml', 'implicit/gbn2.xml')

In [4]:
system = forcefield.createSystem(pdb.topology, soluteDielectric=1.0, solventDielectric=78.5)

In [5]:
integrator = omm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 0.002*unit.picoseconds)

In [6]:
platform = omm.Platform.getPlatformByName('OpenCL')

In [7]:
simulation = app.Simulation(pdb.topology, system, integrator, platform)

In [8]:
simulation.context.setPositions(pdb.positions)

In [9]:
simulation.minimizeEnergy()

In [10]:
h5_reporter = HDF5Reporter('traj.h5', 2500)

In [11]:
stdout_reporter = app.StateDataReporter(stdout, 1000000, step=True,
        potentialEnergy=True, temperature=True)

In [12]:
simulation.reporters.append(h5_reporter)

In [13]:
simulation.reporters.append(stdout_reporter)

In [14]:
%time simulation.step(250000000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000000,-1277.2724609375,301.16882682452183
2000000,-1433.16015625,295.0360108839308
3000000,-1327.765625,275.1995966283314
4000000,-1353.873046875,305.38733744671725
5000000,-1462.30517578125,313.11556906616875
6000000,-1381.20458984375,282.48193659280025
7000000,-1473.1865234375,273.2256722557714
8000000,-1463.77685546875,281.78957552511145
9000000,-1477.2763671875,315.0011976521503
10000000,-1503.763671875,302.38282681958566
11000000,-1492.00537109375,314.6168755894572
12000000,-1393.21533203125,298.5397842610061
13000000,-1386.58251953125,312.8874257321194
14000000,-1311.7490234375,295.829022443646
15000000,-1430.3046875,286.0216679256013
16000000,-1380.345703125,286.8543844178603
17000000,-1493.318359375,267.2696855556436
18000000,-1471.5361328125,317.16564474825543
19000000,-1309.923828125,286.7405611580203
20000000,-1361.73876953125,324.31755235684847
21000000,-1496.06640625,302.04947517930844
22000000,-1452.166015625,287.16

In [15]:
h5_reporter.close()